In [315]:
import wrds
import pandas as pd
import numpy as np
import requests
import json
import os
import tqdm
import pandas_datareader.data as reader
import yfinance as yf

## WRDS Connection

In [207]:
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


## Tickers
Aggregation of all current US stock symbols from NASDAQ, NYSE, AMEX

In [208]:
# read in all_tickers.txt
with open('all_tickers.txt', 'r') as f:
    tickers = f.readlines()
tickers = [x.strip() for x in tickers]

tickers_compiled = tuple(tickers)
#len(tickers_tuple)
#tickers_tuple


## Params
Change the following for automatic data queries.

In [209]:
# CHANGE ME
# DATE RANGE
start_date = ('2015-01-01')
end_date = ('2023-03-01')

In [210]:
# DON'T CHANGE ME
params = {'tickers': tickers_compiled, 'start_date': start_date, 'end_date': end_date}

## YFiance

Price Data provided by Yahoo Finance. WRDS data is incomplete.
https://github.com/ranaroussi/yfinance/wiki/Tickers#parameters

In [329]:
# use yfinance to get data for all monthly returns from start_date to end_date for tickers_compiled
data_yfiance_price_mon = yf.download(tickers_compiled, start=start_date, end=end_date,interval='1mo', group_by='ticker')
#data_yfiance_price_mon

[*********************100%***********************]  7430 of 7430 completed

614 Failed downloads:
- DECAW: Period 'max' is invalid, must be one of ['1d', '5d']
- TALKW: Period 'max' is invalid, must be one of ['1d', '5d']
- OAK: No timezone found, symbol may be delisted
- AFRIW: Period 'max' is invalid, must be one of ['1d', '5d']
- ENERW: Period 'max' is invalid, must be one of ['1d', '5d']
- DPCSW: Period 'max' is invalid, must be one of ['1d', '5d']
- AIBBR: Period 'max' is invalid, must be one of ['1d', '5d']
- SAMAW: Period 'max' is invalid, must be one of ['1d', '5d']
- IGICW: Period 'max' is invalid, must be one of ['1d', '5d']
- PNACR: Period 'max' is invalid, must be one of ['1d', '5d']
- DWACW: Period 'max' is invalid, must be one of ['1d', '5d']
- OHAAW: Period 'max' is invalid, must be one of ['1d', '5d']
- BLNGW: Period 'max' is invalid, must be one of ['1d', '5d']
- TOACW: Period 'max' is invalid, must be one of ['1d', '5d']
- MCAAW: Period 'max' is invalid, must be one o

In [331]:
# Extract 'Adj Close' data for each ticker and combine them into a single DataFrame
adj_close_df = pd.concat([data_yfiance_price_mon[ticker]['Adj Close'].rename(ticker) for ticker in tickers], axis=1)

# Display the resulting DataFrame
adj_close_df

                     A         AA    AAC      AACG    AACI   AACIU  AACIW  \
Date                                                                        
2015-01-01   35.424828  36.622684    NaN  0.059999     NaN     NaN    NaN   
2015-02-01   39.686081  34.610191    NaN  0.049189     NaN     NaN    NaN   
2015-03-01   39.065540  30.288897    NaN  0.049797     NaN     NaN    NaN   
2015-04-01   38.992115  31.461067    NaN  0.051133     NaN     NaN    NaN   
2015-05-01   38.822456  29.304274    NaN  0.059635     NaN     NaN    NaN   
2015-06-01   36.362480  26.196775    NaN  0.070323     NaN     NaN    NaN   
2015-07-01   38.692867  23.189436    NaN  0.042024     NaN     NaN    NaN   
2015-08-01   34.308609  22.202646    NaN  0.029149     NaN     NaN    NaN   
2015-09-01   32.437763  22.766235    NaN  0.030607     NaN     NaN    NaN   
2015-10-01   35.782265  21.045805    NaN  0.048704     NaN     NaN    NaN   
2015-11-01   39.629623  22.059212    NaN  0.065464     NaN     NaN    NaN   

In [337]:
long_df = adj_close_df.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Adj Close')
long_df = long_df[long_df.Ticker != 'index']

In [341]:
data_price_yf = long_df
data_price_yf.rename(columns={'Date': 'date', 'Ticker': 'ticker', 'Adj Close': 'monthly_price'}, inplace=True)
# calculate monthly returns
data_price_yf['monthly_return'] = data_price_yf.groupby('ticker')['monthly_price'].pct_change()
#data_price_yf

/Users/frankzhu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/var/folders/7f/zzvg3mdj425g3y3pzll4q80h0000gn/T/ipykernel_59666/1619042410.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_price_yf['monthly_return'] = data_price_yf.groupby('ticker')['monthly_price'].pct_change()


,date,ticker,monthly_price,monthly_return
98,2015-01-01,A,35.424828,NaN
99,2015-02-01,A,39.686081,0.120290
100,2015-03-01,A,39.065540,-0.015636
101,2015-04-01,A,38.992115,-0.001880
102,2015-05-01,A,38.822456,-0.004351
...,...,...,...,...
728233,2022-10-01,ZYXI,11.400000,0.256891
728234,2022-11-01,ZYXI,13.720000,0.203509
728235,2022-12-01,ZYXI,13.910000,0.013848
728236,2023-01-01,ZYXI,14.130000,0.015816


In [343]:
data_price_yf['date'] = pd.to_datetime(data_price_yf['date'])
data_price_yf['date']=data_price_yf['date'].dt.strftime('%Y-%m')
#data_price_yf

/var/folders/7f/zzvg3mdj425g3y3pzll4q80h0000gn/T/ipykernel_59666/3372563623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_price_yf['date'] = pd.to_datetime(data_price_yf['date'])
/var/folders/7f/zzvg3mdj425g3y3pzll4q80h0000gn/T/ipykernel_59666/3372563623.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_price_yf['date']=data_price_yf['date'].dt.strftime('%Y-%m')


,date,ticker,monthly_price,monthly_return
98,2015-01,A,35.424828,NaN
99,2015-02,A,39.686081,0.120290
100,2015-03,A,39.065540,-0.015636
101,2015-04,A,38.992115,-0.001880
102,2015-05,A,38.822456,-0.004351
...,...,...,...,...
728233,2022-10,ZYXI,11.400000,0.256891
728234,2022-11,ZYXI,13.720000,0.203509
728235,2022-12,ZYXI,13.910000,0.013848
728236,2023-01,ZYXI,14.130000,0.015816


## WRDS Columns

In [211]:
tables = db.list_tables("wrdsapps")
# check whether the table contains m_stock
#print(db.describe_table(library='wrdsapps_finratio', table='firm_ratio'))
#print(tables)
ratio_table = db.describe_table(library='wrdsapps_finratio', table='firm_ratio')

pd.set_option('display.max_rows', 98)
#ratio_table
# print out the value 
# ratio_table

Approximately 2782963 rows in wrdsapps_finratio.firm_ratio.


## Financial Ratios

In [212]:
# Financial Ratios: change the first line in SELECT ...... to include more ratios
# start date and end date can be changed above

data_ratios = db.raw_sql(
    """
    SELECT PERMNO as PERMNO, gvkey as gvkey,TICKER, public_date as date, bm as book_to_market, 
    debt_assets, pe_inc as price_to_earnings, Accrual     
    FROM wrdsapps_finratio.firm_ratio 
    WHERE TICKER IN %(tickers)s AND public_date >= %(start_date)s AND public_date <= %(end_date)s
    """,
params=params)

In [213]:
data_ratios.shape

(256528, 8)

## Firm Info, Price, and More

In [263]:
tables = db.list_tables("crsp_a_stock")

In [264]:
tables

['dse',
 'dseall',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',
 'dsf_v2',
 'dsfhdr',
 'dsi',
 'dsiy',
 'inddlyseriesdata',
 'indfamilyinfohdr',
 'indmthseriesdata',
 'indseriesinfohdr',
 'metacalendarperiod',
 'metacolumncoverage',
 'metacolumninfo',
 'metaexchangecalendar',
 'metafileinfo',
 'metaflagcoverage',
 'metaflaginfo',
 'metaflagtype',
 'metaiteminfo',
 'metasiztociz',
 'mse',
 'mseall',
 'msedelist',
 'msedist',
 'mseexchdates',
 'msenames',
 'msenasdin',
 'mseshares',
 'msf',
 'msf_v2',
 'msfhdr',
 'msi',
 'msiy',
 'saz_agg_ann',
 'saz_agg_mth',
 'saz_agg_qtr',
 'saz_del',
 'saz_dind',
 'saz_dis',
 'saz_dp_dly',
 'saz_ds_dly',
 'saz_hdr',
 'saz_indhdr',
 'saz_mdel',
 'saz_mind',
 'saz_mth',
 'saz_nam',
 'saz_ndi',
 'saz_shr',
 'stkannsecuritydata',
 'stkdelists',
 'stkdistributions',
 'stkdlysecuritydata',
 'stkdlysecurityprimarydata',
 'stkissuerinfohdr',
 'stkissuerinfohist',
 'stkmthfloatshares',
 'stkmthsecuritydata',
 'st

In [265]:
price_table = db.describe_table(library='crsp_a_stock', table='wrds_msfv2_query')


Approximately 4924050 rows in crsp_a_stock.wrds_msfv2_query.


In [266]:
# company monthly stock data
data_price = db.raw_sql(
    """
    SELECT PERMNO as PERMNO,  Ticker as Ticker, MthCalDt as Date, ICBIndustry as ICB_industry_code, MthRet as monthly_return, sprtrn as snp_monthly_return, MthCap as monthly_mktcap, MthPrcVol as monthly_price_volume, ShrOut as shares_outstanding, MthPrc as monthly_price
    FROM crsp_a_stock.wrds_msfv2_query
    WHERE Ticker in %(tickers)s AND MthCalDt >= %(start_date)s AND MthCalDt <= %(end_date)s
    """, 
    params=params)

In [267]:
data_price.shape

(406754, 10)

## Bonds (Rf)

In [216]:
# Bonds
data_bond = db.raw_sql(
    """
    SELECT caldt as date, b10ret as ten_yr_YTM_rf
    FROM crspm.mcti
    WHERE caldt >= %(start_date)s AND caldt <= %(end_date)s
    """,
    params=params)

## ESG Scores
Unfortunately, WRDS does not support queries directly via SQL, so the data was obtained through the WRDS query forms 

In [268]:
# ESG Data Processing
# read in ESG data
esg_score_raw = pd.read_excel('snpESG_all_ex.xlsx')


In [269]:
# get rid of the .0 in gvkey
esg_score_raw['gvkey'] = esg_score_raw['gvkey'].astype(str).str[:-2]

In [270]:
esg_score_raw

,institutionid,Score Date,Aspect Name,Score Value,gvkey,Ticker,Company Name,Incorporation State,State,Zip Code
0,100003,2017-04-19,S&P Global ESG Score,34,24447,BOKF,BOK Financial Corporation,OK,OK,74172
1,100003,2017-04-19,Economic Governance Dimension,50,24447,BOKF,BOK Financial Corporation,OK,OK,74172
2,100003,2017-04-19,Environmental Dimension,23,24447,BOKF,BOK Financial Corporation,OK,OK,74172
3,100003,2017-04-19,Social Dimension,21,24447,BOKF,BOK Financial Corporation,OK,OK,74172
4,100003,2017-04-19,S&P Global ESG Score,23,24447,BOKF,BOK Financial Corporation,OK,OK,74172
...,...,...,...,...,...,...,...,...,...,...
166563,109851650,2023-02-17,Social Dimension,11,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166564,109851650,2023-02-17,S&P Global ESG Score,8,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166565,109851650,2023-02-17,Economic Governance Dimension,20,38015,HLGN,"Heliogen, Inc.",DE,CA,91103
166566,109851650,2023-02-17,Environmental Dimension,0,38015,HLGN,"Heliogen, Inc.",DE,CA,91103


In [271]:
# re-arrange so that company id goes first, and ticker follows, the Score Data, Aspect Name, Socre Value, and institutionid
esg_score_raw = esg_score_raw[['gvkey', 'Company Name', 'Ticker', 'Score Date', 'Aspect Name', 'Score Value', 'institutionid']]
# Movie values of aspect name to columns
esg_score_raw = esg_score_raw.pivot_table(index=['gvkey','Company Name', 'Ticker', 'Score Date', 'institutionid'], columns='Aspect Name', values='Score Value')
# sort by score date
esg_score_raw = esg_score_raw.sort_values(by=['Score Date'])

In [272]:
esg_score_raw = esg_score_raw.sort_values(by=['Score Date'])
esg_score_finished = esg_score_raw.reset_index()
#esg_score_finished.head()

In [273]:
# check whether there are duplicates in esg_score_finished
esg_score_finished.duplicated().sum()
# for fama-french
#fama_french_betas.duplicated().sum()

0

In [274]:
# rename Ticker to ticker, Score Date to date
esg_score_finished = esg_score_finished.rename(columns={'Ticker': 'ticker', 'Score Date': 'date'})
#esg_score_finished.head()

In [275]:
# some cleaning
data_ratios['date'] = pd.to_datetime(data_ratios['date'])

In [276]:
# reformat to YYYY-MM
data_ratios['date'] = data_ratios['date'].dt.strftime('%Y-%m')

In [277]:
data_price['date'] = pd.to_datetime(data_price['date'])

In [278]:
data_price['date'] = data_price['date'].dt.strftime('%Y-%m')

In [344]:
# Merge the DataFrames on 'date' and 'ticker'
data_price_yf_wrds = pd.merge(data_price_yf, data_price, on=['date', 'ticker'], how='outer', suffixes=('_yf', '_price'))

# Fill missing values in 'monthly_return' columns using the data from the other DataFrame
data_price_yf_wrds['monthly_return'] = data_price_yf_wrds['monthly_return_yf'].fillna(data_price_yf_wrds['monthly_return_price'])

# Drop the redundant 'monthly_return' columns
data_price_yf_wrds.drop(columns=['monthly_return_yf', 'monthly_return_price'], inplace=True)

# Display the resulting DataFrame
data_price_yf_wrds

,date,ticker,monthly_price_yf,permno,icb_industry_code,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price_price,shares_turnover,monthly_return
0,2015-01,A,35.424828,87432.0,HEALTH,-0.031041,12687018.54,2.071654e+09,335902.0,37.77,6167.437169,-0.075156
1,2015-02,A,39.686081,87432.0,HEALTH,0.054893,14182560.00,2.126875e+09,336000.0,42.21,6329.985185,0.120290
2,2015-03,A,39.065540,87432.0,HEALTH,-0.017396,13952905.50,1.860486e+09,335810.0,41.55,5540.293195,-0.015636
3,2015-04,A,38.992115,87432.0,HEALTH,0.008521,13892459.70,2.010405e+09,335810.0,41.37,5986.732992,-0.001880
4,2015-05,A,38.822456,87432.0,HEALTH,0.010491,13832013.90,1.854380e+09,335810.0,41.19,5522.110528,-0.004351
...,...,...,...,...,...,...,...,...,...,...,...,...
731671,2022-10,ZYXI,11.400000,18418.0,HEALTH,0.079863,426964.20,3.000368e+07,37453.0,11.40,801.102279,0.256891
731672,2022-11,ZYXI,13.720000,18418.0,HEALTH,0.053753,513855.16,5.955845e+07,37453.0,13.72,1590.218397,0.203509
731673,2022-12,ZYXI,13.910000,18418.0,HEALTH,-0.058971,520971.23,5.742279e+07,37453.0,13.91,1533.195952,0.013848
731674,2023-01,ZYXI,14.130000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015816


In [356]:
# merge data_price and data_ratios
data_price_ratios = pd.merge(data_price_yf_wrds, data_ratios, on=['ticker', 'date', 'permno' ], how='inner')

In [358]:
# delete the last four columns
data_price_ratios = data_price_ratios[data_price_ratios.columns[:-4]]

In [362]:
data_price_ratios = data_price_ratios.drop(columns=['monthly_price_yf'])
data_price_ratios.isnull().sum()
# drop monthly_price_yf column
data_price_ratios.dropna()
data_price_ratios

,date,ticker,permno,icb_industry_code,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price_price,shares_turnover,monthly_return,gvkey
0,2015-01,A,87432.0,HEALTH,-0.031041,12687018.54,2.071654e+09,335902.0,37.77,6167.437169,-0.075156,126554
1,2015-02,A,87432.0,HEALTH,0.054893,14182560.00,2.126875e+09,336000.0,42.21,6329.985185,0.120290,126554
2,2015-03,A,87432.0,HEALTH,-0.017396,13952905.50,1.860486e+09,335810.0,41.55,5540.293195,-0.015636,126554
3,2015-04,A,87432.0,HEALTH,0.008521,13892459.70,2.010405e+09,335810.0,41.37,5986.732992,-0.001880,126554
4,2015-05,A,87432.0,HEALTH,0.010491,13832013.90,1.854380e+09,335810.0,41.19,5522.110528,-0.004351,126554
...,...,...,...,...,...,...,...,...,...,...,...,...
256759,2022-08,ZYXI,18418.0,HEALTH,-0.042440,343358.58,4.528873e+07,38407.0,8.94,1179.179035,0.041958,129441
256760,2022-09,ZYXI,18418.0,HEALTH,-0.093396,339825.69,3.090275e+07,37467.0,9.07,824.799119,0.014541,129441
256761,2022-10,ZYXI,18418.0,HEALTH,0.079863,426964.20,3.000368e+07,37453.0,11.40,801.102279,0.256891,129441
256762,2022-11,ZYXI,18418.0,HEALTH,0.053753,513855.16,5.955845e+07,37453.0,13.72,1590.218397,0.203509,129441


In [282]:
# read in betas
fama_french_betas = pd.read_excel('fama_french_betas.xlsx')
capm_betas = pd.read_excel('capm_beta.xlsx')

In [283]:
data_bond['date'] = pd.to_datetime(data_bond['date'])
data_price_ratios['date'] = pd.to_datetime(data_price_ratios['date'])
esg_score_finished['date'] = pd.to_datetime(esg_score_finished['date'])
capm_betas['date'] = pd.to_datetime(capm_betas['date'])
fama_french_betas['date'] = pd.to_datetime(fama_french_betas['date'])
# in data_ratios, calculate shares_turnover, which is monthly_price_volume/shares_outstanding
data_price['shares_turnover'] = data_price['monthly_price_volume']/data_price['shares_outstanding']

In [284]:
# rename fama-french Ticker Symbol to ticker
fama_french_betas = fama_french_betas.rename(columns={'Ticker Symbol': 'ticker'})

In [285]:
# merge data_price_ratios and esg_score_finished
# change the date column in data_price_ratios and esg_score_finished to year-month-day format
data_price_ratios['date'] = data_price_ratios['date'].dt.strftime('%Y-%m')
esg_score_finished['date'] = esg_score_finished['date'].dt.strftime('%Y-%m')
data_bond['date'] = data_bond['date'].dt.strftime('%Y-%m')
capm_betas['date'] = capm_betas['date'].dt.strftime('%Y-%m')
fama_french_betas['date'] = fama_french_betas['date'].dt.strftime('%Y-%m')

In [286]:
type(esg_score_finished['gvkey'][0])
# change the type of gvkey in esg_score_finished to int
esg_score_finished['gvkey'] = esg_score_finished['gvkey'].astype(str)
# rename fama_french_betas PERMNO to permno
fama_french_betas = fama_french_betas.rename(columns={'PERMNO': 'permno'})
capm_betas = capm_betas.rename(columns={'PERMNO': 'permno'})

## Merging the dataframes

In [354]:
data_price_yf_wrds.dropna()

,date,ticker,monthly_price_yf,permno,icb_industry_code,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price_price,shares_turnover,monthly_return
0,2015-01,A,35.424828,87432.0,HEALTH,-0.031041,12687018.54,2.071654e+09,335902.0,37.77,6167.437169,-0.075156
1,2015-02,A,39.686081,87432.0,HEALTH,0.054893,14182560.00,2.126875e+09,336000.0,42.21,6329.985185,0.120290
2,2015-03,A,39.065540,87432.0,HEALTH,-0.017396,13952905.50,1.860486e+09,335810.0,41.55,5540.293195,-0.015636
3,2015-04,A,38.992115,87432.0,HEALTH,0.008521,13892459.70,2.010405e+09,335810.0,41.37,5986.732992,-0.001880
4,2015-05,A,38.822456,87432.0,HEALTH,0.010491,13832013.90,1.854380e+09,335810.0,41.19,5522.110528,-0.004351
...,...,...,...,...,...,...,...,...,...,...,...,...
731669,2022-08,ZYXI,8.940000,18418.0,HEALTH,-0.042440,343358.58,4.528873e+07,38407.0,8.94,1179.179035,0.041958
731670,2022-09,ZYXI,9.070000,18418.0,HEALTH,-0.093396,339825.69,3.090275e+07,37467.0,9.07,824.799119,0.014541
731671,2022-10,ZYXI,11.400000,18418.0,HEALTH,0.079863,426964.20,3.000368e+07,37453.0,11.40,801.102279,0.256891
731672,2022-11,ZYXI,13.720000,18418.0,HEALTH,0.053753,513855.16,5.955845e+07,37453.0,13.72,1590.218397,0.203509


In [346]:
data_price['monthly_return'].isnull().sum()

202

In [498]:
# merge data_price_ratios and esg_score_finished
data_incomplete = pd.merge(data_price_ratios, esg_score_finished, on=['ticker', 'date','gvkey'], how='right')
# drop company name and institutionid from data_incomplete
data_incomplete = data_incomplete.drop(['Company Name', 'institutionid'], axis=1)
data_complete = pd.merge(data_incomplete, data_bond, on=['date'], how='left')
# merge data_complete_missing_beta and fama_french_betas
# data_complete_missing_capm_beta = pd.merge(data_complete_missing_beta, fama_french_betas, on=['ticker', 'date', 'permno'], how='inner')
# merge data_complete_missing_beta and capm_betas
#data_complete = pd.merge(data_complete_missing_capm_beta, capm_betas, on=['ticker', 'date', 'permno'], how='inner')

In [499]:
merged_data = data_complete.merge(data_price_yf, on=['date', 'ticker'], suffixes=('', '_yf'))

# If 'monthly_return' is null in data_complete, use 'monthly_return' from data_price_yf
merged_data['monthly_return'] = merged_data.apply(lambda row: row['monthly_return_yf'] if pd.isnull(row['monthly_return']) else row['monthly_return'], axis=1)

# Drop the 'monthly_return_yf' column as it's not needed anymore
merged_data = merged_data.drop(columns=['monthly_return_yf'])

data_complete = merged_data

In [500]:
data_price_yf.loc[(data_price_yf['ticker'] == 'SIRI') & (data_price_yf['date'] == '2015-03')]

,date,ticker,monthly_price,monthly_return
597998,2015-03,SIRI,3.67298,-0.017995


In [501]:
# which ticker at which date is missing
data_complete[data_complete['monthly_return'].isnull()]

,date,ticker,permno,icb_industry_code,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price_price,shares_turnover,monthly_return,gvkey,Economic Governance Dimension,Environmental Dimension,S&P Global ESG Score,Social Dimension,ten_yr_ytm_rf,monthly_price
4,2015-03,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1274,44.0,14.0,25.5,16.5,0.008165,NaN
24,2015-03,CBL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29082,50.5,14.5,25.0,19.0,0.008165,NaN
61,2015-03,MEG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185309,42.0,13.0,29.5,28.0,0.008165,NaN
105,2015-03,AGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201535,56.0,23.0,47.0,50.5,0.008165,NaN
138,2015-03,ATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184651,66.5,16.0,41.0,30.0,0.008165,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18340,2023-02,ATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184651,36.5,25.5,30.0,28.0,NaN,NaN
18341,2023-02,ATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,318728,22.0,15.0,19.0,18.0,NaN,NaN
18620,2023-02,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1274,16.0,8.0,11.0,7.0,NaN,NaN
19058,2023-02,ETI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279157,31.0,18.0,28.5,31.0,NaN,NaN


In [502]:
data_complete

,date,ticker,permno,icb_industry_code,snp_monthly_return,monthly_mktcap,monthly_price_volume,shares_outstanding,monthly_price_price,shares_turnover,monthly_return,gvkey,Economic Governance Dimension,Environmental Dimension,S&P Global ESG Score,Social Dimension,ten_yr_ytm_rf,monthly_price
0,2015-01,PKG,87541.0,INDL,-0.031041,7462350.55,1.691286e+09,98383.0,75.85,17190.831486,-0.028187,128978,48.5,14.5,29.5,22.5,0.042437,61.617828
1,2015-01,PKG,87541.0,INDL,-0.031041,7462350.55,1.691286e+09,98383.0,75.85,17190.831486,-0.028187,128978,48.5,14.5,29.5,22.5,0.042437,61.617828
2,2015-01,HUN,90558.0,BASMAT,-0.031041,5362588.08,1.948693e+09,244198.0,21.96,7979.971300,-0.035996,162401,48.5,32.0,38.5,34.0,0.042437,18.064899
3,2015-01,HUN,90558.0,BASMAT,-0.031041,5362588.08,1.948693e+09,244198.0,21.96,7979.971300,-0.035996,162401,48.5,32.0,38.5,34.0,0.042437,18.064899
4,2015-03,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1274,44.0,14.0,25.5,16.5,0.008165,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19319,2023-02,CDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.197943,3153,56.0,23.0,38.0,35.0,NaN,3.120000
19320,2023-02,SHOO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012552,29382,33.0,16.5,23.5,20.5,NaN,36.299999
19321,2023-02,SNEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.147360,31521,38.0,10.0,25.5,16.5,NaN,100.830002
19322,2023-02,CPK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015942,2986,27.0,11.0,16.0,11.5,NaN,128.089996


In [503]:
# print the columns with the most NAs
data_complete.isna().sum().sort_values(ascending=False).head(20)

shares_turnover                  16862
permno                           16862
icb_industry_code                16862
snp_monthly_return               16862
monthly_mktcap                   16862
monthly_price_volume             16862
shares_outstanding               16862
monthly_price_price              16862
ten_yr_ytm_rf                     1405
monthly_return                     536
monthly_price                      517
gvkey                                0
Economic Governance Dimension        0
Environmental Dimension              0
S&P Global ESG Score                 0
Social Dimension                     0
ticker                               0
date                                 0
dtype: int64

In [504]:
# only keep date, ticker, Social Dimension, monthly_return, monthly_price,gvkey,Economic Governance Dimension,Environmental Dimension, S&P Global ESG Score
data_complete = data_complete[['date','gvkey', 'ticker','monthly_return', 'monthly_price', 'gvkey', 'Economic Governance Dimension', 'Environmental Dimension', 'Social Dimension', 'S&P Global ESG Score']]

data_price_ind = data_price[['date','ticker', 'icb_industry_code']]

# merge icb_industry_code into data_complete from data_price based on ticker
data_complete_ind = pd.merge(data_complete, data_price_ind[['date','ticker','icb_industry_code']], on=['ticker','date'], how='left')

In [505]:
data_complete_ind

,date,gvkey,ticker,monthly_return,monthly_price,gvkey,Economic Governance Dimension,Environmental Dimension,Social Dimension,S&P Global ESG Score,icb_industry_code
0,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
1,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
2,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
3,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
4,2015-03,1274,Y,NaN,NaN,1274,44.0,14.0,16.5,25.5,FINL
...,...,...,...,...,...,...,...,...,...,...,...
19477,2023-02,3153,CDE,-0.197943,3.120000,3153,56.0,23.0,35.0,38.0,NaN
19478,2023-02,29382,SHOO,0.012552,36.299999,29382,33.0,16.5,20.5,23.5,NaN
19479,2023-02,31521,SNEX,0.147360,100.830002,31521,38.0,10.0,16.5,25.5,NaN
19480,2023-02,2986,CPK,0.015942,128.089996,2986,27.0,11.0,11.5,16.0,NaN


In [492]:
data_price_ind

,date,ticker,icb_industry_code
0,2015-01,JJSF,CONSTAP
1,2015-01,PLXS,TECH
2,2015-01,RMCF,CONSTAP
3,2015-01,ADX,FINL
4,2015-01,ORCL,TECH
...,...,...,...
406749,2022-12,SIX,CONDIS
406750,2022-12,VPG,INDL
406751,2022-12,FN,TECH
406752,2022-12,SANW,CONSTAP


In [507]:
data_complete_ind.isna().sum().sort_values(ascending=False).head(20)

icb_industry_code                2465
monthly_return                    537
monthly_price                     518
date                                0
gvkey                               0
ticker                              0
gvkey                               0
Economic Governance Dimension       0
Environmental Dimension             0
Social Dimension                    0
S&P Global ESG Score                0
dtype: int64

In [509]:
# drop na
data_complete_ind = data_complete_ind.dropna()
data_complete_ind

,date,gvkey,ticker,monthly_return,monthly_price,gvkey,Economic Governance Dimension,Environmental Dimension,Social Dimension,S&P Global ESG Score,icb_industry_code
0,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
1,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
2,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
3,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
5,2015-03,30662,SIRI,-0.017995,3.672980,30662,36.0,13.5,20.5,26.5,CONDIS
...,...,...,...,...,...,...,...,...,...,...,...
18072,2022-12,65710,EPR,-0.093487,37.720001,65710,35.0,3.0,16.0,16.0,REIT
18073,2022-12,28380,OFIX,0.141190,20.530001,28380,27.0,9.5,15.0,20.0,HEALTH
18074,2022-12,271773,PPL,-0.010163,29.219999,271773,27.5,21.0,35.0,27.5,UTIL
18075,2022-12,11264,WBA,-0.099759,37.360001,11264,58.0,49.0,56.0,54.0,CONSTAP


In [511]:
data_complete = data_complete_ind

In [512]:
# rename 'Economic Governance Dimension', 'Environmental Dimension' 'S&P Global ESG Score', 'Social Dimension' to be without spaces and lowercase
data_complete = data_complete.rename(columns={'Economic Governance Dimension': 'economic_gov_dim', 'Environmental Dimension': 'env_dim', 'S&P Global ESG Score': 'sp_esg_score', 'Social Dimension': 'social_dim'})

#data_complete.columns

## Output Stocks Data

In [513]:
# export to excel
#data_ratios.to_excel('financial_ratios.xlsx', index=False)
#esg_score_finished.to_excel('ESG_Score_Processed.xlsx', index=False)
data_complete.to_excel('data_complete_2.xlsx', index=False)
#data_complete_2021_11.to_excel('data_complete_2021_11.xlsx', index=False)

## Fama-French Factors

In [514]:
# Index by zero here since we only need the first df (monthly instead of yearly)
factors = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start_date, end=end_date)[0]

In [515]:
# some cleaning to prepare for merging
factors = factors.reset_index(inplace=False)
factors.rename(columns={'Date': 'date'}, inplace=True)
#datetime YYYY-MM
factors['date'] = factors['date'].dt.strftime('%Y-%m')

In [516]:
data_complete

,date,gvkey,ticker,monthly_return,monthly_price,gvkey,economic_gov_dim,env_dim,social_dim,sp_esg_score,icb_industry_code
0,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
1,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL
2,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
3,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT
5,2015-03,30662,SIRI,-0.017995,3.672980,30662,36.0,13.5,20.5,26.5,CONDIS
...,...,...,...,...,...,...,...,...,...,...,...
18072,2022-12,65710,EPR,-0.093487,37.720001,65710,35.0,3.0,16.0,16.0,REIT
18073,2022-12,28380,OFIX,0.141190,20.530001,28380,27.0,9.5,15.0,20.0,HEALTH
18074,2022-12,271773,PPL,-0.010163,29.219999,271773,27.5,21.0,35.0,27.5,UTIL
18075,2022-12,11264,WBA,-0.099759,37.360001,11264,58.0,49.0,56.0,54.0,CONSTAP


In [517]:
# Merge factors with data_complete on date
data_complete_ff = pd.merge(data_complete, factors, on='date', how='left')
data_complete_ff

,date,gvkey,ticker,monthly_return,monthly_price,gvkey,economic_gov_dim,env_dim,social_dim,sp_esg_score,icb_industry_code,Mkt-RF,SMB,HML,RF
0,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
1,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
2,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
3,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
4,2015-03,30662,SIRI,-0.017995,3.672980,30662,36.0,13.5,20.5,26.5,CONDIS,-1.12,3.04,-0.37,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16897,2022-12,65710,EPR,-0.093487,37.720001,65710,35.0,3.0,16.0,16.0,REIT,-6.41,-0.68,1.32,0.33
16898,2022-12,28380,OFIX,0.141190,20.530001,28380,27.0,9.5,15.0,20.0,HEALTH,-6.41,-0.68,1.32,0.33
16899,2022-12,271773,PPL,-0.010163,29.219999,271773,27.5,21.0,35.0,27.5,UTIL,-6.41,-0.68,1.32,0.33
16900,2022-12,11264,WBA,-0.099759,37.360001,11264,58.0,49.0,56.0,54.0,CONSTAP,-6.41,-0.68,1.32,0.33


In [518]:
# rename Mkt-RF as risk_premium
data_complete_ff = data_complete_ff.rename(columns={'Mkt-RF': 'risk_premium'})

In [522]:
data_complete_ff

,date,gvkey,ticker,monthly_return,monthly_price,gvkey,economic_gov_dim,env_dim,social_dim,sp_esg_score,icb_industry_code,risk_premium,SMB,HML,RF
0,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
1,2015-01,128978,PKG,-0.028187,61.617828,128978,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
2,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
3,2015-01,162401,HUN,-0.035996,18.064899,162401,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
4,2015-03,30662,SIRI,-0.017995,3.672980,30662,36.0,13.5,20.5,26.5,CONDIS,-1.12,3.04,-0.37,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16897,2022-12,65710,EPR,-0.093487,37.720001,65710,35.0,3.0,16.0,16.0,REIT,-6.41,-0.68,1.32,0.33
16898,2022-12,28380,OFIX,0.141190,20.530001,28380,27.0,9.5,15.0,20.0,HEALTH,-6.41,-0.68,1.32,0.33
16899,2022-12,271773,PPL,-0.010163,29.219999,271773,27.5,21.0,35.0,27.5,UTIL,-6.41,-0.68,1.32,0.33
16900,2022-12,11264,WBA,-0.099759,37.360001,11264,58.0,49.0,56.0,54.0,CONSTAP,-6.41,-0.68,1.32,0.33


In [525]:
# drop gvkey
data_complete_ff = data_complete_ff.drop(['gvkey'], axis=1)
data_complete_ff

KeyError: "['gvkey'] not found in axis"

In [526]:
data_complete_ff

,date,ticker,monthly_return,monthly_price,economic_gov_dim,env_dim,social_dim,sp_esg_score,icb_industry_code,risk_premium,SMB,HML,RF
0,2015-01,PKG,-0.028187,61.617828,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
1,2015-01,PKG,-0.028187,61.617828,48.5,14.5,22.5,29.5,INDL,-3.11,-0.55,-3.58,0.00
2,2015-01,HUN,-0.035996,18.064899,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
3,2015-01,HUN,-0.035996,18.064899,48.5,32.0,34.0,38.5,BASMAT,-3.11,-0.55,-3.58,0.00
4,2015-03,SIRI,-0.017995,3.672980,36.0,13.5,20.5,26.5,CONDIS,-1.12,3.04,-0.37,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16897,2022-12,EPR,-0.093487,37.720001,35.0,3.0,16.0,16.0,REIT,-6.41,-0.68,1.32,0.33
16898,2022-12,OFIX,0.141190,20.530001,27.0,9.5,15.0,20.0,HEALTH,-6.41,-0.68,1.32,0.33
16899,2022-12,PPL,-0.010163,29.219999,27.5,21.0,35.0,27.5,UTIL,-6.41,-0.68,1.32,0.33
16900,2022-12,WBA,-0.099759,37.360001,58.0,49.0,56.0,54.0,CONSTAP,-6.41,-0.68,1.32,0.33


In [527]:
# data_complete_ff.to_excel('data_complete_ff.xlsx', index=False)

## SUR data processing

In [2]:
import pandas as pd

In [3]:
df_sur = pd.read_excel('data_complete_ff.xlsx')

In [ ]:
# 

## Old Code (Ignore everything below)
For record keeping purposes

In [462]:
# DON't mind these data below, just for record keeping
# turn tickers_compiled into a dataframe
#tickers_compiled = pd.DataFrame(tickers_compiled)
# export to txt file
# tickers_compiled.to_excel('tickers_compiled.xlsx')
# VARIABLES 
# variables = ("TICKER", "public_date", "bm", "capital_ratio", "evm", "ptb", "roe")
# LIBRARY
# library = ('wrdsapps_finratio')
# Table
# table = ('firm_ratio')
# date_name
# date_name = ('public_date')
# ticker_name
# ticker_name = ('ticker')
# list tables 
# db.list_tables(library='wrdsapps_finratio')
# Find all the column names in the table
# print(db.describe_table(library='wrdsapps_finratio', table='firm_ratio'))
# Run the following code to see what the schema is for the for any tables in a library
data_test = db.raw_sql(
    """
    SELECT *
    FROM trucost.wrds_esg
    LIMIT 6
    """,
params=params)
data_test.columns()

KeyboardInterrupt: 

In [ ]:
# read in Final_data_yearly.xls

yearly_data = pd.read_excel('Final_data_yearly.xls')


In [ ]:
yearly_data.head()

,CompanyName,ticker,year,institutionid,EconomicGovernanceDimension,EnvironmentalDimension,SPGlobalESGScore,SocialDimension,PERMNO,gvkey,...,b_mkt_fama_french_3fac1,b_smb_fama_french_3fac1,b_hml_fama_french_3fac1,alpha_fama_french_3fac1,excess_return_fama_french_3fac1,permno,icb_industry_code,mktcap,yearly_return,industry
0,"Agilent Technologies, Inc.",A,2019,4075849,51.0,75.0,53.0,50.0,87432,126554,...,4.321428,-16.811880,-87.152542,0.021524,-2.575978,87432,HEALTH,23936570.0,0.275302,6
1,"Agilent Technologies, Inc.",A,2020,4075849,47.0,80.0,51.0,50.0,87432,126554,...,-0.433493,4.535038,13.811031,0.030218,-0.400570,87432,HEALTH,28794336.0,0.397681,6
2,"Agilent Technologies, Inc.",A,2021,4075849,50.0,77.0,55.0,55.0,87432,126554,...,-7.571819,-9.820291,-2.614265,0.027274,0.522970,87432,HEALTH,43736736.0,0.280805,6
3,American Airlines Group Inc.,AAL,2019,4065191,43.5,40.0,37.5,29.0,21020,1045,...,3.606888,16.296570,70.999405,-0.003790,1.802358,21020,CONDIS,13631192.0,-0.095702,2
4,American Airlines Group Inc.,AAL,2020,4065191,57.0,48.0,53.0,52.0,21020,1045,...,38.878998,115.990257,136.770126,-0.029870,-0.242799,21020,CONDIS,6849860.0,-0.448205,2


In [ ]:
# check the shape of the data
yearly_data.shape


(3007, 30)

In [ ]:
# only getting the 2021 data
yearly_data_2021 = yearly_data[yearly_data['year'] == 2021]

In [ ]:
## Regression
# first, add a constant in our matrix
df_spread = sm.add_constant(df_spread)
df_spread.head()

# run the regression
model = sm.OLS(df_spread['Ri-Rf'], df_spread[['const', 'Rm-Rf', 'SMB', 'HML', 'ESG']])

# get the results
results = model.fit()

## Results
print(results.summary())

NameError: name 'df_spread' is not defined

In [ ]:
yearly_data_2021.columns

Index(['CompanyName', 'ticker', 'year', 'institutionid',
       'EconomicGovernanceDimension', 'EnvironmentalDimension',
       'SPGlobalESGScore', 'SocialDimension', 'PERMNO', 'gvkey',
       'book_to_market1', 'capital_ratio1', 'enterprise_value_multiple1',
       'price_to_book1', 'return_on_equity1', 'inventory_current_asset_frac1',
       'receivable_current_asset_frac1', 'cash_ratio1', 'effective_tax_rate1',
       'debt_assets1', 'b_mkt_fama_french_3fac1', 'b_smb_fama_french_3fac1',
       'b_hml_fama_french_3fac1', 'alpha_fama_french_3fac1',
       'excess_return_fama_french_3fac1', 'permno', 'icb_industry_code',
       'mktcap', 'yearly_return', 'industry'],
      dtype='object')

In [ ]:
# run a regression 
# import statsmodels.api as sm
# With mktcap, book_to_market1, b_mkt_fama_french_3fac1, SPGlobalESGScore, and interaction between icb_industry_code and SPGlobalESGScore
# dependent variable is yearly_return

import statsmodels.api as sm
import statsmodels.formula.api as smf

# run a regression
# with mktcap, book_to_market1, b_mkt_fama_french_3fac1, SPGlobalESGScore, and interaction between icb_industry_code and SPGlobalESGScore
# dependent variable is yearly_return
model = smf.ols(formula='yearly_return ~ b_mkt_fama_french_3fac1 + mktcap + book_to_market1 + SPGlobalESGScore + icb_industry_code:SPGlobalESGScore', data=yearly_data_2021)
results = model.fit()


# output the results and export to pdf
print(results.summary())
results.summary().as_latex()








                            OLS Regression Results                            
Dep. Variable:          yearly_return   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     4.619
Date:                Tue, 06 Dec 2022   Prob (F-statistic):           3.40e-08
Time:                        23:26:04   Log-Likelihood:                -648.63
No. Observations:                1023   AIC:                             1327.
Df Residuals:                    1008   BIC:                             1401.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

'\\begin{center}\n\\begin{tabular}{lclc}\n\\toprule\n\\textbf{Dep. Variable:}                                  &  yearly\\_return  & \\textbf{  R-squared:         } &     0.060   \\\\\n\\textbf{Model:}                                          &       OLS        & \\textbf{  Adj. R-squared:    } &     0.047   \\\\\n\\textbf{Method:}                                         &  Least Squares   & \\textbf{  F-statistic:       } &     4.619   \\\\\n\\textbf{Date:}                                           & Tue, 06 Dec 2022 & \\textbf{  Prob (F-statistic):} &  3.40e-08   \\\\\n\\textbf{Time:}                                           &     23:26:04     & \\textbf{  Log-Likelihood:    } &   -648.63   \\\\\n\\textbf{No. Observations:}                               &        1023      & \\textbf{  AIC:               } &     1327.   \\\\\n\\textbf{Df Residuals:}                                   &        1008      & \\textbf{  BIC:               } &     1401.   \\\\\n\\textbf{Df Model:}          

In [ ]:
# correlation matrix
corr = yearly_data_2021.corr()
corr


,year,institutionid,EconomicGovernanceDimension,EnvironmentalDimension,SPGlobalESGScore,SocialDimension,PERMNO,gvkey,book_to_market1,capital_ratio1,...,debt_assets1,b_mkt_fama_french_3fac1,b_smb_fama_french_3fac1,b_hml_fama_french_3fac1,alpha_fama_french_3fac1,excess_return_fama_french_3fac1,permno,mktcap,yearly_return,industry
year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
institutionid,NaN,1.000000,-0.109392,-0.097317,-0.126178,-0.105919,-0.160954,0.069424,-0.192981,0.033026,...,-0.172350,0.001993,-0.014498,0.019058,-0.107894,-0.006077,-0.163092,-0.027111,-0.091929,0.033858
EconomicGovernanceDimension,NaN,-0.109392,1.000000,0.789936,0.947631,0.874908,0.039614,-0.072421,-0.018654,0.089893,...,0.082176,-0.011481,-0.027052,-0.045333,0.097875,0.019361,0.038998,0.174648,0.061595,0.044167
EnvironmentalDimension,NaN,-0.097317,0.789936,1.000000,0.908678,0.823878,0.021244,-0.079478,-0.026349,0.043969,...,0.075695,-0.005590,-0.014104,-0.027889,0.054784,0.009729,0.019628,0.243138,0.038901,0.033697
SPGlobalESGScore,NaN,-0.126178,0.947631,0.908678,1.000000,0.955253,0.022313,-0.074000,-0.036655,0.076998,...,0.108053,-0.003195,-0.015596,-0.043093,0.078306,0.009186,0.021598,0.219112,0.052780,0.058015
SocialDimension,NaN,-0.105919,0.874908,0.823878,0.955253,1.000000,0.003750,-0.058830,-0.051459,0.071059,...,0.113107,-0.003496,-0.005308,-0.037753,0.073740,0.007437,0.003137,0.200116,0.049506,0.059538
PERMNO,NaN,-0.160954,0.039614,0.021244,0.022313,0.003750,1.000000,0.309406,0.032435,0.013625,...,-0.018561,0.017882,-0.011320,0.001865,0.079663,-0.013900,0.997370,-0.024296,0.037808,-0.071390
gvkey,NaN,0.069424,-0.072421,-0.079478,-0.074000,-0.058830,0.309406,1.000000,-0.008682,-0.016462,...,-0.019738,0.017421,-0.013099,0.049504,0.009454,-0.022276,0.307672,-0.039061,0.012686,0.028296
book_to_market1,NaN,-0.192981,-0.018654,-0.026349,-0.036655,-0.051459,0.032435,-0.008682,1.000000,-0.151096,...,0.122166,0.016397,0.019262,0.050196,0.156465,-0.015157,0.034194,-0.094374,0.116504,-0.160652
capital_ratio1,NaN,0.033026,0.089893,0.043969,0.076998,0.071059,0.013625,-0.016462,-0.151096,1.000000,...,0.265256,0.463599,-0.010169,-0.152095,0.243419,-0.435807,0.012990,0.001349,-0.020754,-0.032782


In [ ]:
data_complete.columns

Index(['Company Name', 'PERMNO', 'ticker', 'date', 'icb_industry_code',
       'monthly_return', 'snp_monthly_return', 'monthly_mktcap',
       'monthly_price_volume', 'gvkey', 'book_to_market', 'capital_ratio',
       'enterprise_value_multiple', 'price_to_book', 'return_on_equity',
       'inventory_current_asset_frac', 'receivable_current_asset_frac',
       'cash_ratio', 'effective_tax_rate', 'debt_assets',
       'Economic Governance Dimension', 'Environmental Dimension',
       'S&P Global ESG Score', 'Social Dimension', 'ten_yr_ytm_rf',
       'b_mkt_fama_french_3fac', 'b_smb_fama_french_3fac',
       'b_hml_fama_french_3fac', 'alpha_fama_french_3fac',
       'excess_return_fama_french_3fac', 'b_mkt_capm_mkt', 'alpha_capm',
       'excess_return_capm'],
      dtype='object')

In [ ]:
# read in data_complete.xlsx

data_complete = pd.read_excel('data_complete.xlsx')
data_analysis = pd.read_excel('Final_data_yearly.xls')
data_complete.rename(columns={'permno': 'PERMNO'}, inplace=True)
# join the b_mkt_capm_mkt from data_complete to data_analysis on PERMNO and gvkey
data_analysis_beta = data_analysis.merge(data_complete[['PERMNO', 'gvkey', 'b_mkt_capm_mkt']], on=['PERMNO', 'gvkey'], how='left')


FileNotFoundError: [Errno 2] No such file or directory: 'data_complete.xlsx'

In [ ]:
data_analysis_beta.shape
data_analysis_beta.to_excel('data_analysis_beta.xlsx')

In [ ]:
# for company with the same year and same PERMNO and gvkey, take the average of all the variables
data_complete_monthly = data_analysis_beta.groupby(['PERMNO', 'gvkey', 'date']).mean().reset_index()